In [415]:
import numpy as np
import torch 
import torchvision
from torchvision.transforms import ToTensor, Normalize
from sklearn.model_selection import train_test_split
from abc import ABC, abstractmethod
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score

# Part II : Multinominal Softmax Classification

In [436]:
raw_mnist = torchvision.datasets.MNIST("./", download=True)

x_train, x_test = train_test_split(raw_mnist.data, test_size=0.3, random_state=166003)
y_train, y_test = train_test_split(raw_mnist.targets, test_size=0.3, random_state=166003)

x_train = x_train.reshape(-1, 28 * 28).numpy()
x_test = x_test.reshape(-1, 28 * 28).numpy()

# important so softmax wont overflow 
x_train = (x_train - np.min(x_train)) / (np.max(x_train) - np.min(x_train))
x_test = (x_test - np.min(x_test)) / (np.max(x_test) - np.min(x_test))


y_test = y_test.numpy()
y_train = y_train.numpy()

# one hotting labels
# y_test = np.eye(10)[y_test].astype(np.int32)
y_train = np.eye(10)[y_train].astype(np.int32)

print(x_train.shape, x_test.shape, y_train.shape, y_test.shape)


(42000, 784) (18000, 784) (42000, 10) (18000,)


## Gradients 


Feed forward:
$$ z = \hat{y}= \textbf{xw} + \textbf{b} $$

Softmax Activation Function:

$$\sigma (z) = \frac{\text{exp}(z_i)}{\sum_{j=0}^{K}\text{exp}(z_j)}$$
$$\sigma (w) = \frac{\text{exp}(xw_i+b_i)}{\sum_{j=0}^{K}\text{exp}(xw_j+b_j)}$$

Loss Function - CrossEntropy for Binary Classification: 

$$ L(y, \hat{y}) = −log \hspace{2px} p(y|x) = -[y \hspace{2px}log \hspace{2px} \hat{y} \hspace{2px}+\hspace{2px} (1-y)\hspace{2px}log \hspace{2px}(1-\hat{y})]$$

Considering the labels are represented in one-hot fashion, in which each element of $\hat{y}$ represents the probability $ p(\hat{y_k}= 1 |x)$, therefore the multiclass version will be the sum of the logs of the $K$ classes 

$$ L(y, \hat{y}) = -\sum_{i=0}^{K} y_i \hspace{2px}log\hspace{2px}\hat{y_i}  $$ 

Note that, given the one hot encoding all terms that are not the desired class k, will be equal to zero, simplifying the loss to: 

$$ L(y, \hat{y}) = -\hspace{2px}log\hspace{2px}\hat{y_k}  $$

$$ L(w) = -log\hspace{2px} \left(  \frac{\text{exp}(\textbf{xw}_i + \textbf{b}_i )}{\sum_{j=0}^{K}\text{exp}(\textbf{xw}_j + \textbf{b}_j )}\right) $$


$$L(z) = -log\hspace{2px} \left(  \frac{e^{z_i}} {\sum_{j=0}^{K}e^{z_j}} \right) $$

By applying the log division rule, this will be helpfull to avoid using the derivative quotient rule :

$$L(z) = -\left( z_i -log\sum_{j=0}^{K}e^{z_j}  \right)$$

Now computing the derivative in respect to the class $i$ :

$$ \frac{\partial L}{\partial z_i} = -\frac{\partial }{\partial z_i} \left( z_i -log\sum_{j=0}^{K}e^{z_j}  \right) $$

$$\frac{\partial L}{\partial z_i} = \frac{\partial }{\partial z_i} z_i - \frac{\partial }{\partial z_i}log\sum_{j=0}^{K}e^{z_j}   $$

$$\frac{\partial L}{\partial z_i} = 1 - log\sum_{j=0}^{K}\frac{\partial }{\partial z_i}e^{z_j} $$

Note that the derivative for all the possibilities except for the desired class, its zero, therefore:

$$ \frac{\partial }{\partial z_i}e^{z_j} = \left\{ \begin{array}{cl}
0 & : \ z_j \neq z_i \\
e^{z_i} & : \ z_j  = z_i
\end{array} \right. $$

$$ \frac{\partial }{\partial z_i}log\left( \sum_{j=0}^{K}e^{z_j} \right) = \frac{1}{\sum_{j=0}^{K}e^{z_j} } e^{z_i} 
 $$

$$\frac{\partial L}{\partial z_i} = 1 - \frac{1}{\sum_{j=0}^{K}e^{z_j} } e^{z_i} = 1 - z_i$$


Now applying the chain rule: 

$$  \frac{\partial L}{\partial w_i} = \frac{\partial L}{\partial z_i} \frac{\partial z_i}{\partial w_i}     $$

$$  \frac{\partial L}{\partial b_i} = \frac{\partial L}{\partial z_i} \frac{\partial z_i}{\partial b_i}     $$

$$ \frac{\partial L}{\partial w_i} =  \left( 1 -  z_i \right) \textbf{x}  =  \left( 1 - \frac{\text{exp}(\textbf{xw}_i + \textbf{b}_i )}{\sum_{j=0}^{K}\text{exp}(\textbf{xw}_j + \textbf{b}_j )}\right)\textbf{x}$$ 

$$ \frac{\partial L}{\partial b_i} =  \left( 1 -  z_i \right)  =  \left( 1 - \frac{\text{exp}(\textbf{xw}_i + \textbf{b}_i )}{\sum_{j=0}^{K}\text{exp}(\textbf{xw}_j + \textbf{b}_j )}\right)$$ 

In [ ]:
class SoftmaxRegression():
    def __init__(self, p_size, n_classes, lr=0.001):
        # self.w = np.random.rand(p_size, n_classes) * 0.01
        self.w = np.random.rand(n_classes, p_size) * 0.01
        self.b = np.random.rand(n_classes)
        self.lr = lr
        self.v_w = np.zeros_like(self.w)
        self.v_b = np.zeros_like(self.b)

    def forward(self, X):
        # return ( X @ self.w ) + self.b 
        return ( X @ self.w.T ) + self.b 


    def compute_probs(self, logits):
        # softmax the network outputs
        # return np.exp(logits) / np.sum(np.exp(logits))
        exp_logits = np.exp(logits - np.max(logits, axis=1, keepdims=True))  # Stability trick
        return exp_logits / np.sum(exp_logits, axis=1, keepdims=True)

    def crossEntropy(self, y_hat, y):
        b = - y * np.log(y_hat)
        return b.sum(axis=-1) / len(y)

    def loss_derivative(self, x, y_hat, y):
        # return ( 1 - y_hat ).T @ x / len(x), np.sum(1 - y_hat, axis=0) / len(x)
        return ( y - y_hat).T @ x / len(x), np.sum(y - y_hat, axis=0) / len(x)

    
    def gradient_descent(self, w_gradient, b_gradient):
        
        self.w -= self.lr * w_gradient
        self.b -= self.lr * b_gradient

    def stochastic_gradient_descent(self, X, y):
        
        random_indexes = np.arange(len(X))
        np.random.shuffle(random_indexes)

        X = X[random_indexes]
        y = y[random_indexes]
        
        logits = self.forward(X)
        y_hat = self.compute_probs(logits)
        ce_loss = self.crossEntropy(y_hat, y)
        w_gradient, b_gradient = self.loss_derivative(X, y_hat) 

        self.w -= self.lr * w_gradient.T
        self.b -= self.lr * b_gradient.T

        return ce_loss

    def momentum_sgd(self, X, y, beta=0.9):
        
        random_indexes = np.arange(len(X))
        np.random.shuffle(random_indexes)

        X = X[random_indexes]
        y = y[random_indexes]
        
        logits = self.forward(X)
        y_hat = self.compute_probs(logits)
        ce_loss = self.crossEntropy(y_hat, y)
        w_gradient, b_gradient = self.loss_derivative(X, y_hat) 

        self.v_w = self.v_w * beta - self.lr * w_gradient.T
        self.v_b = self.v_b * beta - self.lr * b_gradient.T

        self.w -= self.v_w
        self.b -= self.v_b 

        return ce_loss
    
    def optimize_model(self, opt_strategy, epochs, traindata, testdata):
    
        x_train, y_train = traindata
        x_test, y_test = testdata

        loss_record = []
        opt_foo = self.gradient_descent        
        # if opt_strategy == "GD":
        #     opt_foo = self.gradient_descent
        # elif opt_strategy == "SGD":
        #     opt_foo = self.stochastic_gradient_descent

        # elif opt_strategy == "M-SGD":
        #     opt_foo = self.momentum_sgd

        for e in range(epochs):

            a = self.w

            logits = self.forward(x_train)
            y_hat = self.compute_probs(logits)
            ce_loss = self.crossEntropy(y_hat, y_train)

            w_gradient, b_gradient = self.loss_derivative(x_train, y_hat, y_train) 
            opt_foo(w_gradient, b_gradient)
            
            preds = np.argmax(self.compute_probs(self.forward(x_test)), axis=1)
            print(preds[0])
            print("acc =", accuracy_score(y_test, preds) )

            print(ce_loss.sum())


In [ ]:
clf = SoftmaxRegression(784, 10, lr=0.1)

clf.optimize_model("GD", 25, (x_train, y_train), (x_test, y_test))

1
2.339057442251065
1
2.480771517140523
1
2.6646585406916494
1
2.9338911821738116
1
3.4047921211541436
1
4.385542898987815
1
6.444204902052498
1
9.57836398254892
1
13.045716774863395
1
16.566757671195973
1
20.09613885165557
1
23.626975369381153
1
27.158100924354663
1
30.689290739540063
1
34.22049626388289
1
37.751705947689096
1
41.282916809373724
1
44.81412802393851
1
48.3453393493052
1
51.87655071084495
1
55.40776208458106
1
58.938973462540794
1
62.47018484199615
1
66.0013962219911
1
69.53260760218382


In [ ]:
import numpy as np

class MultinomialLogisticRegression:
    def __init__(self, n_features, n_classes, learning_rate=0.01):
        self.n_features = n_features
        self.n_classes = n_classes
        self.learning_rate = learning_rate
        self.weights = np.random.randn(n_classes, n_features) * 0.01
        self.biases = np.zeros(n_classes)
    
    def softmax(self, logits):
        """
        Computes softmax probabilities for the input logits.
        """
        exp_logits = np.exp(logits - np.max(logits, axis=1, keepdims=True))  # Stability trick
        return exp_logits / np.sum(exp_logits, axis=1, keepdims=True)
    
    def forward(self, X):
        """
        Performs the forward pass, computing logits and softmax probabilities.
        """
        logits = np.dot(X, self.weights.T) + self.biases
        probabilities = self.softmax(logits)
        return logits, probabilities
    
    def compute_loss(self, y_true, probabilities):
        """
        Computes cross-entropy loss.
        y_true: Ground truth labels (one-hot encoded).
        probabilities: Softmax probabilities.
        """
        n_samples = y_true.shape[0]
        log_probs = np.log(probabilities + 1e-15)
        loss = -np.sum(y_true * log_probs) / n_samples
        return loss
    
    def backward(self, X, y_true, probabilities):
        """
        Computes gradients of weights and biases.
        """
        n_samples = X.shape[0]
        error = probabilities - y_true  # Gradient of loss with respect to logits
        grad_weights = np.dot(error.T, X) / n_samples
        grad_biases = np.sum(error, axis=0) / n_samples
        return grad_weights, grad_biases
    
    def update_parameters(self, grad_weights, grad_biases):
        """
        Updates weights and biases using gradient descent.
        """
        self.weights -= self.learning_rate * grad_weights
        self.biases -= self.learning_rate * grad_biases
    
    def train(self, X, y, epochs=100):
        """
        Trains the model for a given number of epochs.
        """
        for epoch in range(epochs):
            logits, probabilities = self.forward(X)
            loss = self.compute_loss(y, probabilities)
            grad_weights, grad_biases = self.backward(X, y, probabilities)
            self.update_parameters(grad_weights, grad_biases)
            if (epoch + 1) % 10 == 0:
                print(f"Epoch {epoch + 1}/{epochs}, Loss: {loss:.4f}")
    
    def predict(self, X):
        """
        Predicts class labels for input samples.
        """
        _, probabilities = self.forward(X)
        return np.argmax(probabilities, axis=1)


# # Example usage
# if __name__ == "__main__":
#     # Simulate a dataset
#     np.random.seed(42)
#     n_samples = 150
#     n_features = 4
#     n_classes = 3

#     # Random input data
#     X = np.random.rand(n_samples, n_features)

#     # Random one-hot encoded labels
#     y_indices = np.random.randint(0, n_classes, n_samples)
#     y = np.eye(n_classes)[y_indices]  # One-hot encoding

#     # Initialize and train the model
model = MultinomialLogisticRegression(784, 10, learning_rate=0.1)
model.train(x_train, y_train, epochs=100)

#     # Predict and evaluate
#     predictions = model.predict(X)
#     accuracy = np.mean(predictions == y_indices)
#     print(f"Training Accuracy: {accuracy * 100:.2f}%")


Epoch 10/100, Loss: 1.5964
Epoch 20/100, Loss: 1.2114
Epoch 30/100, Loss: 1.0083
